<a href="https://colab.research.google.com/github/keulreobeu/deep_learning_for_the_life_sciences/blob/main/%ED%98%84%EB%AF%B8%EA%B2%BD%EC%9D%84_%EC%9C%84%ED%95%9C_%EB%94%A5%EB%9F%AC%EB%8B%9D_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [1]:
!pip install --pre deepchem

런타임 재시작 후 다시실행


In [1]:
!pip install "keras<3" "tensorflow<2.16"

런타임 재시작 후 다시실행

In [2]:
!pip install rdkit

In [3]:
!pip list | egrep 'deepchem|rdkit|tensorflow|keras|numpy|pandas|scikit-learn|matplotlib'


deepchem                              2.8.1.dev20250829211407
geopandas                             1.0.1
keras                                 2.15.0
keras-hub                             0.18.1
keras-nlp                             0.18.1
matplotlib                            3.10.0
matplotlib-inline                     0.1.7
matplotlib-venn                       1.1.2
numpy                                 1.26.4
pandas                                2.2.2
pandas-datareader                     0.10.0
pandas-gbq                            0.29.2
pandas-stubs                          2.2.2.240909
rdkit                                 2025.3.6
scikit-learn                          1.6.1
sklearn-pandas                        2.2.0
tensorflow                            2.15.1
tensorflow-datasets                   4.9.9
tensorflow_decision_forests           1.11.0
tensorflow-estimator                  2.15.0
tensorflow-hub                        0.16.1
tensorflow-io-gcs-filesystem         

In [4]:
import deepchem as dc
from rdkit import Chem
import numpy as np
import pandas as pd

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


### 구버전 코드
```
# 이미지 다운
wget https://data.broadinstitute.org/bbbc/BBBC005/BBBC005_v1_images.zip
unzip BBBC005_v1_images.zip


# 이미지 로드
image_dir = 'BBBC005_v1_images'
files = []
labels = []

for f in os.listdir(image_dir):
  if f.endswith('.tif'):
    files.append(os.path.join(image_dir, f))
    labels.append(int(re.findall('_c(.*?)_', f)[0]))

loaber = dc.data.ImageLoader()
dataset = loaber.featurize(files, np.array(labels))


# test, valid 분리
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset, seed = 123)


# 모델 생성
iearning_rate = dc.models.tensorgraph.optimizers.ExponentiaDecay(
    0.001, 0.9, 250)
model = dc.models.TensorGraph(learning_rate = learning_rate, model_dir = 'model')
features = layers.Features(shape=(None,))
labels = layers.Labels(shape=(None,))

for num_outputs in [16, 32, 64, 128, 256]:
    prev_layer = layers.Conv2D(num_outputs, kernel_size=5, stride=2, in_layers=prev_layer)

outputs = laters.Dens(1, in_layers=prev_layer.Flatten(prev_layer))
model.add_outputs(outputs)
loss = layers.ReduceSum(layers.L2Loss(in_layers=(outputs, labels)))
model.set_loss(loss)


# 모델의 결과 가져오기 모델 학습에는 1시간 이상걸림
model.restore()


# 모델 평가
y_pred = model.predict(test_dataset).flatten()
print(np.sprt(np.mean((y_pred-test_dataset.y)**2)))
```

In [9]:
print('런타임 유지')

런타임 유지


In [12]:
# =========================
# 0) 기본 설정 & 드라이브
# =========================
import os, re, math, random
import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split

# Colab에서만: 주석 해제해서 사용
from google.colab import drive
drive.mount('/content/drive')

# --- 경로 설정 (필요에 맞게 바꿔주세요) ---
# 학습된 모델 ZIP을 드라이브에 두었다면, 먼저 Colab에서 압축 해제:
# !unzip -o "/content/drive/MyDrive/microscopy_models.zip" -d "/content/drive/MyDrive/"

MODEL_ROOT = "/content/drive/MyDrive/data/microscopy_models"   # ZIP을 풀어놓은 루트
CKPT_DIR   = os.path.join(MODEL_ROOT, "model")         # 분류 체크포인트 폴더 (또는 "segmentation")
# CKPT_DIR = os.path.join(MODEL_ROOT, "segmentation")  # 세그멘테이션 평가 시 이 줄로 변경

# BBBC005 이미지 폴더
IMAGE_DIR  = "/content/drive/MyDrive/data/BBBC005_v1_images"

# 이미지 전처리 하이퍼파라미터
IMG_SIZE   = 128  # 필요시 96/128/256 등으로 조정

# 재현성
SEED = 123
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# TF1 호환 모드
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

# === 1) 이미지 찾기: .tif/.tiff 대소문자 무시 + 하위폴더 재귀 ===
tif_files = []
for root, _, files in os.walk(IMAGE_DIR):
    for f in files:
        if f.lower().endswith((".tif", ".tiff")):
            tif_files.append(os.path.join(root, f))

print(f"Discovered {len(tif_files)} TIFF files")
print("Sample files:", [os.path.basename(p) for p in tif_files[:5]])
assert len(tif_files) > 0, f"No TIFF files found under {IMAGE_DIR} (check path or unzip status)."

# === 2) 라벨 파싱: '_c숫자_' 또는 '_C숫자_' 모두 허용 ===
def extract_label_from_name(fname: str):
    # 예: A01_c3_s1.TIF -> 3  / A01_C2_s4.TIF -> 2
    m = re.search(r"_[cC](\d+)_", os.path.basename(fname))
    if not m:
        raise ValueError(f"Cannot parse label with pattern '_[cC](\\d+)_' from filename: {os.path.basename(fname)}")
    return int(m.group(1))

labels = [extract_label_from_name(p) for p in tif_files]
classes = sorted(set(labels))
num_classes = len(classes)
print(f"Classes: {classes} (num_classes={num_classes})")

# === 3) 이미지 로딩/전처리 (그레이스케일 → 리사이즈 → [0,1]) ===
IMG_SIZE = 128  # 학습 당시 입력 크기와 맞추세요
def load_and_preprocess(path):
    img = Image.open(path).convert("L")
    img = img.resize((IMG_SIZE, IMG_SIZE), Image.BILINEAR)
    arr = np.asarray(img, dtype=np.float32) / 255.0
    return np.expand_dims(arr, -1)

X = np.stack([load_and_preprocess(p) for p in tif_files], axis=0)
y = np.array(labels, dtype=np.int32)
print("X shape:", X.shape, " y shape:", y.shape)

# 간단 분할(20% 테스트)
X_trv, X_te, y_trv, y_te = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y if num_classes > 1 else None
)
print("Test shape:", X_te.shape)

# =========================
# 2) TF1 체크포인트 로드 유틸
# =========================
def latest_ckpt_prefix(ckpt_dir):
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        return ckpt.model_checkpoint_path
    # fallback: 가장 큰 global_step 가진 prefix 찾기
    metas = [f for f in os.listdir(ckpt_dir) if f.endswith(".meta")]
    assert metas, f"No .meta found in {ckpt_dir}"
    steps = []
    for m in metas:
        # e.g., "model-7700.meta" -> "model-7700"
        prefix = m[:-5]
        try:
            step = int(prefix.split("-")[-1])
        except:
            step = -1
        steps.append((step, prefix))
    steps.sort()
    return os.path.join(ckpt_dir, steps[-1][1])

def list_placeholders_and_outputs(graph):
    ops = graph.get_operations()
    placeholders = [op.name for op in ops if op.type == "Placeholder"]
    # 흔한 출력 후보 탐색: Softmax, Sigmoid, Identity, ArgMax 등
    candidates = [op.name for op in ops if op.type in ("Softmax","Sigmoid","Identity","ArgMax")]
    return placeholders, candidates

def guess_io_tensors(graph):
    placeholders, candidates = list_placeholders_and_outputs(graph)
    in_guess = None
    # 가장 그럴듯한 입력 플레이스홀더 추정(이름/shape 기준)
    for name in placeholders:
        try:
            t = graph.get_tensor_by_name(name + ":0")
            shp = t.shape.as_list()
            if len(shp) == 4:      # (None, H, W, C)
                in_guess = name + ":0"
                break
        except:
            pass
    if in_guess is None and placeholders:
        in_guess = placeholders[0] + ":0"

    # 출력 후보는 뒤쪽에 나오는 걸 우선
    out_guess = None
    for name in reversed(candidates):
        if "softmax" in name.lower() or "prob" in name.lower() or "output" in name.lower():
            out_guess = name + ":0"
            break
    if out_guess is None and candidates:
        out_guess = candidates[-1] + ":0"
    return in_guess, out_guess, placeholders, candidates

# =========================
# 3) 복원 & 평가
# =========================
ckpt_prefix = latest_ckpt_prefix(CKPT_DIR)
print("Restoring from:", ckpt_prefix)

tf.compat.v1.reset_default_graph()
saver = tf.compat.v1.train.import_meta_graph(ckpt_prefix + ".meta", clear_devices=True)
graph = tf.compat.v1.get_default_graph()

# 입력/출력 텐서 자동 추정
input_name, output_name, phs, outs = guess_io_tensors(graph)
print("Placeholder candidates (first 10):", phs[:10])
print("Output op candidates (last 10):", outs[-10:])
print("Auto-guessed INPUT:", input_name)
print("Auto-guessed OUTPUT:", output_name)

# 필요시 아래 두 줄을 원하는 텐서 이름으로 "수동" 지정해 덮어쓰기 가능
# input_name  = "Features:0"
# output_name = "Softmax:0"

x_tensor = graph.get_tensor_by_name(input_name)
y_tensor = graph.get_tensor_by_name(output_name)

with tf.compat.v1.Session(config=config) as sess:
    saver.restore(sess, ckpt_prefix)

    # 배치 처리(메모리 아낄 때)
    def predict_in_batches(X, batch=64):
        outs = []
        for i in range(0, len(X), batch):
            outs.append(sess.run(y_tensor, feed_dict={x_tensor: X[i:i+batch]}))
        return np.concatenate(outs, axis=0)

    y_pred_raw = predict_in_batches(X_te)

# =========================
# 4) 성능 지표 출력
# =========================
if num_classes > 1:
    # 분류: 소프트맥스/로짓 모두 대응
    if y_pred_raw.ndim == 2 and y_pred_raw.shape[1] > 1:
        y_pred = np.argmax(y_pred_raw, axis=1)
    else:
        # 이진 로짓/시그모이드 같은 경우
        y_pred = (y_pred_raw.reshape(-1) >= 0.5).astype(np.int32)
    acc = (y_pred == y_te).mean()
    print(f"[CLASSIFICATION] Test Accuracy: {acc:.4f}")
else:
    # 회귀
    y_pred = y_pred_raw.reshape(-1)
    rmse = math.sqrt(np.mean((y_pred - y_te.astype(np.float32))**2))
    print(f"[REGRESSION] Test RMSE: {rmse:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Discovered 19200 TIFF files
Sample files: ['SIMCEPImages_J01_C1_F29_s07_w1.TIF', 'SIMCEPImages_J01_C1_F29_s15_w2.TIF', 'SIMCEPImages_J02_C5_F29_s07_w1.TIF', 'SIMCEPImages_J02_C5_F29_s03_w1.TIF', 'SIMCEPImages_J01_C1_F29_s12_w2.TIF']
Classes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 31, 35, 40, 44, 48, 53, 57, 61, 66, 70, 74, 78, 83, 87, 91, 96, 100] (num_classes=42)
X shape: (19200, 128, 128, 1)  y shape: (19200,)
Test shape: (3840, 128, 128, 1)
Restoring from: /content/drive/MyDrive/data/microscopy_models/model/model-7700
Placeholder candidates (first 10): ['Placeholder', 'Feature_13_pre_q/Placeholder', 'Label_16_pre_q/Placeholder']
Output op candidates (last 10): ['Conv2D_6/bias/Adam_1/read', 'Conv2D_4/kernel/Adam/read', 'Conv2D_4/kernel/Adam_1/read', 'Conv2D_4/bias/Adam/read', 'Conv2D_4/bias/Adam_1/read', 

ValueError: Cannot feed value of shape (64, 128, 128, 1) for Tensor Placeholder:0, which has shape ()

In [14]:
# ===== 패치: 올바른 입력 텐서를 자동 선택해서 예측/평가 =====
import numpy as np, math, tensorflow as tf
tf.compat.v1.disable_eager_execution()
graph = tf.compat.v1.get_default_graph()

def shape_list(t):
    try:
        return t.shape.as_list()
    except:
        return None

def default_extra_feeds(graph):
    feeds = {}
    for op in graph.get_operations():
        if op.type == "Placeholder":
            t = graph.get_tensor_by_name(op.name + ":0")
            shp = t.shape.as_list()
            if shp == [] or shp == [None] or shp == ():
                name = op.name.lower()
                if t.dtype == tf.bool and ("is_training" in name or "training" in name or "train" in name):
                    feeds[t] = False
                elif t.dtype.is_floating and ("keep_prob" in name or "dropout" in name or "drop_rate" in name):
                    feeds[t] = 1.0
    return feeds

def find_feed_tensor(graph):
    # 1) tf.data / Queue 계열 출력 먼저
    for op in graph.get_operations():
        if op.type in ("IteratorGetNext","QueueDequeueV2","QueueDequeueManyV2","QueueDequeueUpToV2","QueueDequeue"):
            for t in op.outputs:
                shp = shape_list(t)
                if shp and len(shp) == 4:
                    return t
    # 2) 첫 Conv2D의 입력(보통 (N,H,W,C))
    for op in graph.get_operations():
        if op.type == "Conv2D":
            t = op.inputs[0]
            shp = shape_list(t)
            if shp and len(shp) == 4:
                return t
    # 3) 4D Placeholder
    for op in graph.get_operations():
        if op.type == "Placeholder":
            t = graph.get_tensor_by_name(op.name + ":0")
            shp = shape_list(t)
            if shp and len(shp) == 4:
                return t
    return None

def find_output_tensor(graph):
    # Softmax > Sigmoid > Identity(세이버 제외) > BiasAdd
    def last(types):
        for op in reversed(graph.get_operations()):
            if op.type in types:
                return graph.get_tensor_by_name(op.name + ":0")
        return None
    y = last(["Softmax"])
    if y is not None: return y
    y = last(["Sigmoid"])
    if y is not None: return y
    y = last(["Identity"])
    if y is not None and "save" not in y.name.lower():
        return y
    return last(["BiasAdd"])

x_tensor = find_feed_tensor(graph)
y_tensor = find_output_tensor(graph)
assert x_tensor is not None, "입력 텐서를 찾지 못했습니다. (Conv2D 입력 또는 QueueDequeue/Iterator 출력 탐색 실패)"
assert y_tensor is not None, "출력 텐서를 찾지 못했습니다."

print("[Picked INPUT ]", x_tensor.name, "shape=", shape_list(x_tensor))
print("[Picked OUTPUT]", y_tensor.name,  "shape=", shape_list(y_tensor))

extra_feeds = default_extra_feeds(graph)
if extra_feeds:
    print("Extra scalar placeholders auto-fed:",
          {k.name: v for k, v in extra_feeds.items()})

def predict_in_batches(sess, X, batch=64):
    outs = []
    for i in range(0, len(X), batch):
        fd = {x_tensor: X[i:i+batch], **extra_feeds}
        outs.append(sess.run(y_tensor, feed_dict=fd))
    return np.concatenate(outs, axis=0)

with tf.compat.v1.Session(config=config) as sess:
    saver.restore(sess, ckpt_prefix)  # ckpt_prefix는 이전 셀에서 구한 값 사용
    y_pred_raw = predict_in_batches(sess, X_te, batch=64)

# --- 성능 계산: 출력이 [None,1]이면 회귀로 간주 ---
if y_pred_raw.ndim == 2 and y_pred_raw.shape[1] == 1:
    y_pred = y_pred_raw.reshape(-1)
    rmse = math.sqrt(np.mean((y_pred - y_te.astype(np.float32))**2))
    mae  = np.mean(np.abs(y_pred - y_te.astype(np.float32)))
    print(f"[REGRESSION] Test RMSE: {rmse:.4f} | MAE: {mae:.4f}")
else:
    # 다중분류 가정
    if y_pred_raw.ndim == 2 and y_pred_raw.shape[1] > 1:
        y_pred = np.argmax(y_pred_raw, axis=1)
    else:
        y_pred = (y_pred_raw.reshape(-1) >= 0.5).astype(np.int32)
    acc = (y_pred == y_te).mean()
    print(f"[CLASSIFICATION] Test Accuracy: {acc:.4f}")


[Picked INPUT ] Conv2D_12/ExpandDims:0 shape= [None, 520, 696, 1]
[Picked OUTPUT] Dense_1/Dense_1/BiasAdd:0 shape= [None, 1]


ValueError: Cannot feed value of shape (64, 128, 128, 1) for Tensor Conv2D_12/ExpandDims:0, which has shape (None, 520, 696, 1)

In [13]:
# === 교체용: IO 자동탐지 + 평가 ===
import math, numpy as np, tensorflow as tf
tf.compat.v1.disable_eager_execution()
graph = tf.compat.v1.get_default_graph()

def tensor_shape_str(t):
    try:
        return str(t.shape.as_list())
    except:
        return "unknown"

def list_placeholders(graph):
    ph = []
    for op in graph.get_operations():
        if op.type == "Placeholder":
            t = graph.get_tensor_by_name(op.name + ":0")
            ph.append((op.name, t.dtype.name, t.shape.as_list()))
    return ph

def pick_input_placeholder(graph, img_size=128):
    """(N,H,W,C) 형태의 Placeholder 우선, 없으면 2D(플랫텐)도 고려"""
    cands = list_placeholders(graph)
    # 1) rank 4 후보
    rank4 = [(n,d,s) for (n,d,s) in cands if s is not None and len([x for x in s if x is not None])>=1 and len(s)==4]
    # H/W가 img_size와 맞는지 우선 고려
    rank4_sorted = sorted(rank4, key=lambda x: (
        0 if (x[2][1] in (img_size,None) and x[2][2] in (img_size,None)) else 1
    ))
    if rank4_sorted:
        return rank4_sorted[0][0] + ":0"
    # 2) rank 2 후보(플랫텐)
    rank2 = [(n,d,s) for (n,d,s) in cands if s is not None and len(s)==2]
    if rank2:
        return rank2[0][0] + ":0"
    # 3) 최후: 첫 Placeholder
    return cands[0][0] + ":0" if cands else None

def find_output_tensor(graph):
    """Softmax > Sigmoid > Identity > BiasAdd 순으로 후보 탐색"""
    def last_tensor_by_type(types):
        for op in reversed(graph.get_operations()):
            if op.type in types:
                return graph.get_tensor_by_name(op.name + ":0")
        return None
    y = last_tensor_by_type(["Softmax"])
    if y is not None: return y
    y = last_tensor_by_type(["Sigmoid"])
    if y is not None: return y
    # 예측 헤드가 Identity인 경우
    y = last_tensor_by_type(["Identity"])
    if y is not None and "save" not in y.name and "Assign" not in y.name:
        return y
    # Dense의 출력은 보통 BiasAdd
    y = last_tensor_by_type(["BiasAdd"])
    return y

def default_extra_feeds(graph):
    """스칼라 플레이스홀더 자동 feed: is_training=False, dropout/keep_prob=1.0 등"""
    feeds = {}
    for op in graph.get_operations():
        if op.type == "Placeholder":
            t = graph.get_tensor_by_name(op.name + ":0")
            shp = t.shape.as_list()
            if shp == [] or shp == [None] or shp == ():
                name_low = op.name.lower()
                if t.dtype == tf.bool:
                    if "is_training" in name_low or "training" in name_low or "train" in name_low:
                        feeds[t] = False
                elif t.dtype.is_floating:
                    if "keep_prob" in name_low or "dropout" in name_low or "drop_rate" in name_low:
                        feeds[t] = 1.0
                elif t.dtype.is_integer:
                    # 학습 단계/글로벌스텝 등은 굳이 feed 안 함
                    pass
    return feeds

# --- 입력/출력 이름 자동 선택 ---
input_name = pick_input_placeholder(graph, img_size=IMG_SIZE)
y_tensor   = find_output_tensor(graph)
assert input_name is not None, "입력 Placeholder를 찾지 못했습니다."
assert y_tensor is not None,   "출력 텐서를 찾지 못했습니다."

x_tensor = graph.get_tensor_by_name(input_name)

# 디버그: 후보 프린트
print("[Pick] INPUT =", x_tensor.name, "shape=", tensor_shape_str(x_tensor))
print("[Pick] OUTPUT=", y_tensor.name,  "shape=", tensor_shape_str(y_tensor))

# 추가 feed 기본값
extra_feeds = default_extra_feeds(graph)
if extra_feeds:
    print("Extra scalar placeholders auto-fed:", {t.name: v for t,v in extra_feeds.items()})

# --- 예측 실행 ---
def predict_in_batches(sess, X, batch=64):
    outs = []
    for i in range(0, len(X), batch):
        fd = {x_tensor: X[i:i+batch], **extra_feeds}
        outs.append(sess.run(y_tensor, feed_dict=fd))
    return np.concatenate(outs, axis=0)

with tf.compat.v1.Session(config=config) as sess:
    saver.restore(sess, ckpt_prefix)
    y_pred_raw = predict_in_batches(sess, X_te, batch=64)

# --- 성능 계산 ---
if num_classes > 1:
    # 다중분류: softmax 확률 또는 로짓 가정
    if y_pred_raw.ndim == 2 and y_pred_raw.shape[1] > 1:
        y_pred = np.argmax(y_pred_raw, axis=1)
    else:
        # 이진(확률/로짓) 가정
        y_pred = (y_pred_raw.reshape(-1) >= 0.5).astype(np.int32)
    acc = (y_pred == y_te).mean()
    print(f"[CLASSIFICATION] Test Accuracy: {acc:.4f}")
else:
    y_pred = y_pred_raw.reshape(-1)
    rmse = math.sqrt(np.mean((y_pred - y_te.astype(np.float32))**2))
    print(f"[REGRESSION] Test RMSE: {rmse:.4f}")


[Pick] INPUT = Placeholder:0 shape= []
[Pick] OUTPUT= Dense_1/Dense_1/BiasAdd:0 shape= [None, 1]


ValueError: Cannot feed value of shape (64, 128, 128, 1) for Tensor Placeholder:0, which has shape ()

In [ ]:
# =========================
# 3) 복원 & 평가
# =========================
ckpt_prefix = latest_ckpt_prefix(CKPT_DIR)
print("Restoring from:", ckpt_prefix)

tf.compat.v1.reset_default_graph()
saver = tf.compat.v1.train.import_meta_graph(ckpt_prefix + ".meta", clear_devices=True)
graph = tf.compat.v1.get_default_graph()

# 입력/출력 텐서 자동 추정
input_name, output_name, phs, outs = guess_io_tensors(graph)
print("Placeholder candidates (first 10):", phs[:10])
print("Output op candidates (last 10):", outs[-10:])
print("Auto-guessed INPUT:", input_name)
print("Auto-guessed OUTPUT:", output_name)

# 필요시 아래 두 줄을 원하는 텐서 이름으로 "수동" 지정해 덮어쓰기 가능
# input_name  = "Features:0"
# output_name = "Softmax:0"

x_tensor = graph.get_tensor_by_name(input_name)
y_tensor = graph.get_tensor_by_name(output_name)

with tf.compat.v1.Session(config=config) as sess:
    saver.restore(sess, ckpt_prefix)

    # 배치 처리(메모리 아낄 때)
    def predict_in_batches(X, batch=64):
        outs = []
        for i in range(0, len(X), batch):
            outs.append(sess.run(y_tensor, feed_dict={x_tensor: X[i:i+batch]}))
        return np.concatenate(outs, axis=0)

    y_pred_raw = predict_in_batches(X_te)

# =========================
# 4) 성능 지표 출력
# =========================
if num_classes > 1:
    # 분류: 소프트맥스/로짓 모두 대응
    if y_pred_raw.ndim == 2 and y_pred_raw.shape[1] > 1:
        y_pred = np.argmax(y_pred_raw, axis=1)
    else:
        # 이진 로짓/시그모이드 같은 경우
        y_pred = (y_pred_raw.reshape(-1) >= 0.5).astype(np.int32)
    acc = (y_pred == y_te).mean()
    print(f"[CLASSIFICATION] Test Accuracy: {acc:.4f}")
else:
    # 회귀
    y_pred = y_pred_raw.reshape(-1)
    rmse = math.sqrt(np.mean((y_pred - y_te.astype(np.float32))**2))
    print(f"[REGRESSION] Test RMSE: {rmse:.4f}")


In [18]:
# === A CODE: Inference-only with existing TF1.x checkpoint ===
# Colab/TF2.15에서 TF1 그래프 복원하여 추론
# - 모델 텐서명을 모를 때를 대비해 자동 탐색 + 요약 프린트
# - 입력: (None, 520, 696, 1) float32 [0,1] 정규화
# - 라벨: 파일명 "_C{num}_"에서 {num}을 정수로 파싱 (세포 수 회귀)
# - 출력: RMSE

import os, re, glob, json, math
import numpy as np
from PIL import Image
import imageio.v3 as iio
import tensorflow as tf

# ---- 0) 환경설정: TF1 모드로 전환 & GPU 메모리 증가 허용
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()
gpus = tf.config.experimental.list_physical_devices('GPU')
for g in gpus:
    try:
        tf.config.experimental.set_memory_growth(g, True)
    except:
        pass

# ---- 1) 경로 및 하이퍼파라미터
IMAGE_DIR = "/content/drive/MyDrive/data/BBBC005_v1_images"
MODEL_DIR = "/content/drive/MyDrive/data/microscopy_models"
IMG_H, IMG_W, IMG_C = 520, 696, 1

# ---- 2) 유틸: 파일명에서 라벨(세포 수) 파싱
def parse_label_from_name(fname: str) -> int:
    # 예: SIMCEPImages_A01_C1_F1_s11_w2.tif -> 1
    m = re.search(r"_C(\d+)_", fname)
    if not m:
        raise ValueError(f"파일명에서 라벨(C<num>)을 찾을 수 없음: {fname}")
    return int(m.group(1))

# ---- 3) 유틸: TIFF 로더 (흑백 1채널로, 520x696로 리사이즈, [0,1] 스케일)
def load_image(path: str) -> np.ndarray:
    # Pillow로 로드 (TIFF 지원), L = 8bit grayscale
    img = Image.open(path).convert("L")
    if (img.size[1], img.size[0]) != (IMG_H, IMG_W):
        img = img.resize((IMG_W, IMG_H), resample=Image.BILINEAR)
    arr = np.array(img, dtype=np.float32)
    arr = arr / 255.0
    arr = arr[..., None]  # (H, W, 1)
    return arr

# ---- 4) 평가용 서브셋 만들기 (예: 무작위 3개 혹은 더 크게)
all_files = sorted([os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR) if f.lower().endswith(".tif")])
assert len(all_files) > 0, "이미지 폴더에 .tif 파일이 없습니다."
# 무작위 3개 샘플 (원하면 n_sample 늘리세요)
rng = np.random.default_rng(42)
idxs = rng.choice(len(all_files), size=min(64, len(all_files)), replace=False)  # 64장으로 RMSE 한번 계산
sample_files = [all_files[i] for i in idxs]

X = np.stack([load_image(p) for p in sample_files], axis=0)  # (N, 520, 696, 1)
y = np.array([parse_label_from_name(os.path.basename(p)) for p in sample_files], dtype=np.float32)

print("샘플 배치:", X.shape, y.shape, "예) 첫 5개 라벨:", y[:5])

# ---- 5) 체크포인트 탐색 (가장 최신 .meta 기준)
meta_list = sorted(glob.glob(os.path.join(MODEL_DIR, "*.meta")))
if not meta_list:
    raise FileNotFoundError("모델 디렉터리에 .meta 파일을 찾지 못했습니다.")
meta_path = meta_list[-1]
ckpt_prefix = meta_path[:-5]  # .meta 제거
print("가장 최신 메타그래프:", meta_path)
print("체크포인트 prefix:", ckpt_prefix)

# ---- 6) 그래프 로드 & 리스토어
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
saver = tf.compat.v1.train.import_meta_graph(meta_path, clear_devices=True)
saver.restore(sess, ckpt_prefix)

graph = tf.compat.v1.get_default_graph()

# ---- 7) 입력/출력 텐서 자동 추정
def find_input_tensor():
    # (None, 520, 696, 1) Placeholder를 우선 탐색
    candidates = []
    for op in graph.get_operations():
        if op.type == "Placeholder":
            t = op.outputs[0]
            shp = t.shape.as_list()
            if len(shp) == 4 and shp[0] is None and shp[1] == IMG_H and shp[2] == IMG_W and shp[3] == IMG_C:
                candidates.append(t)
    if candidates:
        return candidates[0]
    # 못찾으면 첫 Placeholder 반환
    for op in graph.get_operations():
        if op.type == "Placeholder":
            return op.outputs[0]
    return None

def find_output_tensor():
    # 흔한 이름, 컬렉션, 말단 연산 등을 우선 시도
    common_names = ["predictions:0", "prediction:0", "output:0", "logits:0", "y_pred:0", "y_hat:0", "Identity:0"]
    for name in common_names:
        try:
            t = graph.get_tensor_by_name(name)
            return t
        except:
            pass
    # 말단에 가까운 Sigmoid/Identity/Dense 출력 유사 텐서 추정
    tail_ops = []
    for op in graph.get_operations():
        if op.type in ("Identity", "Sigmoid", "Relu", "Tanh", "BiasAdd", "Add", "MatMul"):
            # 손실/그래디언트 노이즈를 피해 보기 위해 소비자 수가 적은 텐서 선호
            if len(op.outputs[0].consumers()) == 0 or all(c.type.startswith("Gradients") for c in op.outputs[0].consumers()):
                tail_ops.append(op)
    if tail_ops:
        # 가장 마지막 생성된 연산의 출력을 후보로
        return tail_ops[-1].outputs[0]
    # 실패 시 None
    return None

x_tensor = find_input_tensor()
yhat_tensor = find_output_tensor()

if x_tensor is None or yhat_tensor is None:
    # 디버깅을 위해 그래프 요약 출력
    print("\n[디버깅] 그래프에 있는 Placeholder (상위 10개):")
    cnt = 0
    for op in graph.get_operations():
        if op.type == "Placeholder":
            print(" -", op.name, op.outputs[0].shape)
            cnt += 1
            if cnt >= 10: break
    print("\n[디버깅] 샘플 말단 연산 (상위 20개):")
    tail_ops = [op for op in graph.get_operations() if op.type in ("Identity","Sigmoid","Relu","Tanh","BiasAdd","Add","MatMul")]
    for op in tail_ops[-20:]:
        print(" -", op.name, "->", op.outputs[0].name, op.outputs[0].shape)
    raise RuntimeError("입력/출력 텐서를 자동으로 식별하지 못했습니다. 위 요약을 참고해 텐서명을 지정해 주세요.")

print("입력 텐서:", x_tensor.name, x_tensor.shape)
print("출력 텐서:", yhat_tensor.name, yhat_tensor.shape)

# ---- 8) 추론 & 평가 (RMSE)
feed = {x_tensor: X}
y_pred = sess.run(yhat_tensor, feed_dict=feed).squeeze()
y_pred = np.array(y_pred).reshape(-1)

rmse = float(np.sqrt(np.mean((y_pred - y) ** 2)))
print(f"▶ Inference RMSE on {len(y)} images: {rmse:.4f}")
print("예측 상위 10개:", y_pred[:10])


샘플 배치: (64, 520, 696, 1) (64,) 예) 첫 5개 라벨: [44. 14. 57. 48. 66.]


FileNotFoundError: 모델 디렉터리에 .meta 파일을 찾지 못했습니다.